In [50]:
pip install emoji --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import emoji
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers import Embedding
from keras.callbacks import ReduceLROnPlateau

%matplotlib inline


In [162]:
train_data = pd.read_csv('train_emoji.csv', header = None)

In [163]:
test_data = pd.read_csv('test_emoji.csv', header = None)

In [24]:
train_data.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [164]:
train_data.drop([2, 3], axis = 1, inplace = True)


In [26]:
train_data.head()

,0,1
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [27]:
train_data.isnull().sum()

0    0
1    0
dtype: int64

In [165]:
test_data.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a very nice raise\t,2
3,she got me a nice present\t,2
4,ha ha ha it was so funny\t,2


In [166]:
test_data[0]  = test_data[0].map(lambda x : x.replace('\t',''))

In [167]:
test_data.head()

,0,1
0,I want to eat,4
1,he did not answer,3
2,he got a very nice raise,2
3,she got me a nice present,2
4,ha ha ha it was so funny,2


In [31]:
test_data.isnull().sum()

0    0
1    0
dtype: int64

In [168]:
x_train = train_data[0]
y_train1 = train_data[1]
x_test = test_data[0]
y_test1 = test_data[1]

In [170]:
y_train1.unique()

array([3, 2, 0, 4, 1], dtype=int64)

In [171]:
y_train1[100]

0

In [36]:
emoji_dictionary = {"0": ":red_heart:",    
                    "1": ":baseball:",
                    "2": ':smiling_face_with_smiling_eyes:',
                    "3": ":disappointed_face:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    return emoji.emojize(emoji_dictionary[str(label)])

In [38]:
print(x_train[100], label_to_emoji(y_train[100])) # Testing the function

I love my dad ❤️


In [57]:
maxWord = len(max(x_train, key= len).split()) #key is the number of iterations
maxWord

10

In [172]:
#Converting targets to one-hot encoding
y_train = pd.get_dummies(y_train1)
y_test = pd.get_dummies(y_test1)

In [173]:
# with open('glove.6B.100d.txt', encoding="utf8") as file:
# #     file.readline()
#     words = set()        # ensures unique values
#     word_to_vec_map = {}  # this will be a dictionary mapping words to their vectors
#     for l in file:
#         line = l.strip().split()
#         curr_word = line[0]
#         words.add(curr_word)
#         word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
#     i = 1
#     words_to_index = {}   # dictionary mapping words to their index in the dictionary
#     index_to_words = {}   # dictionary mapping index to the word in the dictionary
#     for w in sorted(words):
#         words_to_index[w] = i
#         index_to_words[i] = w
#         i = i + 1
            
    
        
#     print(word_to_vec_map) # Remove character from each line and split
#         curr_word = line[0] # Assign first index of the splited line
#         words.add(curr_word) # Add the first index to the empty set()
#         word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

In [144]:
# # print(word_to_vec_map.get('the'))
# print(words_to_index['love'])


226278


### Global Vectors for Word Representation
GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.

In [150]:
# we will be using 100 d word vectors

def read_glove_vecs(glove_file):
    with open(glove_file, encoding="utf8") as f: # I used encoding cos reading the file was returning error
        
        words = set()         # ensures unique values
        word_to_vec_map = {}  # this will be a dictionary mapping words to their vectors
        for line in f:
            line = line.strip().split() # Remove character from the begining and ending of each line and spliting each item by spaces
            curr_word = line[0] # Assign first index of the splited line
            words.add(curr_word) # Add the first index to the empty set()
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64) # Form a dictionary using the first index as key and others as value
        
        i = 1
        words_to_index = {}   # dictionary mapping words to their index in the dictionary
        index_to_words = {}   # dictionary mapping index to the word in the dictionary
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

words_to_index, index_to_words, word_to_vec_map = read_glove_vecs('glove.6B.100d.txt')



### We now converts each array of sentences into an array of indices with a dictionary containing the each word mapped to its index.

In [147]:
# X_indices = np.zeros((x_train.shape[0], 10))



# for items in range(x_train.shape[0]):                               
        
#         # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
#         sentence_words = x_train[items].lower().split() # We have converted each sentence to lowercase and tokenized
# # print(sentence_words)
# # print(x_train[131])
        
#         j = 0
        
#         # Loop over the words of sentence_words
#         for w in sentence_words:
#             X_indices[i, j] = words_to_index[w]
#             j = j+1  
    

In [148]:
X_indices[0,1]

0.0

In [151]:
def sentences_to_indices(X, word_to_index, max_len):
 
    m = X.shape[0]                                  
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = X[i].lower().split()
        
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            X_indices[i, j] = words_to_index[w]
            j = j+1  
    
    return X_indices

X_train_indices = sentences_to_indices(x_train, words_to_index, maxWord)

In [152]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1               # +1 for Keras  
    emb_dim = 100                                     # dimensionality of your GloVe word vectors
    
    emb_matrix = np.zeros((vocab_len, emb_dim))      # Initialization with zeros
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    
    # Build the embedding layer
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [157]:
def Emojify(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(shape=input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)
    
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(5, activation='softmax')(X)
    X = Activation('softmax')(X)    
    
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

emojifier = Emojify((maxWord,), word_to_vec_map, words_to_index)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=0.00001, verbose=1)
emojifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
emojifier.fit(X_train_indices, y_train, epochs = 50, batch_size = 16, shuffle=True, 
                               callbacks=[reduce_lr])


Epoch 1/50
9/9 [==============================] - 7s 24ms/step - loss: 1.5942 - accuracy: 0.2879 - lr: 0.0010
Epoch 2/50
9/9 [==============================] - 0s 20ms/step - loss: 1.5595 - accuracy: 0.3030 - lr: 0.0010
Epoch 3/50
9/9 [==============================] - 0s 19ms/step - loss: 1.5526 - accuracy: 0.3258 - lr: 0.0010
Epoch 4/50
9/9 [==============================] - 0s 24ms/step - loss: 1.5279 - accuracy: 0.3258 - lr: 0.0010
Epoch 5/50
9/9 [==============================] - 0s 20ms/step - loss: 1.4713 - accuracy: 0.4848 - lr: 0.0010
Epoch 6/50
9/9 [==============================] - 0s 20ms/step - loss: 1.4002 - accuracy: 0.5076 - lr: 0.0010
Epoch 7/50
9/9 [==============================] - 0s 20ms/step - loss: 1.3269 - accuracy: 0.6061 - lr: 0.0010
Epoch 8/50
9/9 [==============================] - 0s 20ms/step - loss: 1.3098 - accuracy: 0.6061 - lr: 0.0010
Epoch 9/50
9/9 [==============================] - 0s 20ms/step - loss: 1.1902 - accuracy: 0.7500 - lr: 0.0010
Epoch 10/5

In [160]:
X_test_indices = sentences_to_indices(x_test, words_to_index, maxWord)

loss, acc = emojifier.evaluate(X_test_indices, y_test)
print()
print("Test accuracy = ", acc)

2/2 [==============================] - 2s 12ms/step - loss: 1.0320 - accuracy: 0.8750

Test accuracy =  0.875


In [194]:
pred = emojifier.predict(X_test_indices)
print(pred[0])

for i in range(len(x_test)):
    
    num = np.argmax(pred[i])
    if(num != y_test1[i]):
        print('Expected emoji:'+ label_to_emoji(y_test1[i]) + ' prediction: '+ x_test[i] + label_to_emoji(num))

2/2 [==============================] - 0s 14ms/step
[0.14889567 0.14891921 0.14887488 0.14887415 0.40443614]
Expected emoji:😊 prediction: he got a very nice raise❤️
Expected emoji:😊 prediction: she got me a nice present❤️
Expected emoji:😞 prediction: work is horrible😊
Expected emoji:😊 prediction: you brighten my day😞
Expected emoji:😞 prediction: she is a bully❤️
Expected emoji:😊 prediction: will you be my valentine❤️
Expected emoji:😞 prediction: go away⚾


In [193]:

for i in range(len(x_test)):
      
    num = np.argmax(pred[i])
    
    print('Expected emoji:'+ label_to_emoji(y_test1[i]) + ' prediction: '+ x_test[i] + label_to_emoji(num))
    

Expected emoji:🍴 prediction: I want to eat🍴
Expected emoji:😞 prediction: he did not answer😞
Expected emoji:😊 prediction: he got a very nice raise❤️
Expected emoji:😊 prediction: she got me a nice present❤️
Expected emoji:😊 prediction: ha ha ha it was so funny😊
Expected emoji:😊 prediction: he is a good friend😊
Expected emoji:😞 prediction: I am upset😞
Expected emoji:😊 prediction: We had such a lovely dinner tonight😊
Expected emoji:🍴 prediction: where is the food🍴
Expected emoji:😊 prediction: Stop making this joke ha ha ha😊
Expected emoji:⚾ prediction: where is the ball⚾
Expected emoji:😞 prediction: work is hard😞
Expected emoji:😞 prediction: This girl is messing with me😞
Expected emoji:😞 prediction: are you serious😞
Expected emoji:⚾ prediction: Let us go play baseball⚾
Expected emoji:😞 prediction: This stupid grader is not working 😞
Expected emoji:😞 prediction: work is horrible😊
Expected emoji:😊 prediction: Congratulation for having a baby😊
Expected emoji:😞 prediction: stop pissing me off😞

In [195]:
import pickle

In [202]:
# container = open('emoji.pickle', 'wb')
# pickle.dump(emojifier, container)
# container.close()


In [201]:
# container_opener = open('emoji.pickle', 'rb')
# emojifier = pickle.load(container_opener)
# container_opener.close()